In [1]:
import sagemaker

In [69]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
from sagemaker import image_uris

In [38]:
bucket_name="app-1090"
my_region = boto3.session.Session().region_name
print(my_region)

eu-north-1


In [39]:
s3 = boto3.resource("s3")
try:
    if my_region =="eu-north-1":
         s3.create_bucket(Bucket="app-1090",CreateBucketConfiguration={'LocationConstraint':'eu-north-1'})
    print("s3 bucket created successfully!")
except Exception as e:
    print("S3 error",e)
        

s3 bucket created successfully!


In [41]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format("app-1090", prefix)
print(output_path)

s3://app-1090/xgboost-as-a-built-in-algo/output


In [42]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
     

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [43]:
import numpy as np
train_data, test_data =np.split(model_data.sample(frac=1,random_state=1729),[int(0.7 * len(model_data))])
print(train_data.shape,test_data.shape)

(28831, 61) (12357, 61)


In [50]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket("app-1090").Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format("app-1090", prefix), content_type='csv')

In [53]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket("app-1090").Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format("app-1090", prefix), content_type='csv')

# building the model

In [ ]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='1.0-1')

In [ ]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [76]:
from sagemaker import image_uris
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name,version ="latest")


estimator = sagemaker.estimator.Estimator(image_uri=container,
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_type = 'ml.m5.2xlarge',
                                          instance_count= 1,
                                          volume_size = 5,
                                          output_path=output_path,
                                           use_spot_instances=True,
                                           max_run=300,
                                           max_wait= 600)



INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [77]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: xgboost-2023-09-22-08-10-23-112


2023-09-22 08:10:23 Starting - Starting the training job...
2023-09-22 08:10:40 Starting - Preparing the instances for training......
2023-09-22 08:11:38 Downloading - Downloading input data...
2023-09-22 08:12:13 Training - Training image download completed. Training in progress....
2023-09-22 08:12:44 Uploading - Uploading generated training modelArguments: train
[2023-09-22:08:12:35:INFO] Running standalone xgboost training.
[2023-09-22:08:12:35:INFO] File size need to be processed in the node: 2.9mb. Available memory size in the node: 23947.08mb
[2023-09-22:08:12:35:INFO] Determined delimiter of CSV input is ','
[08:12:35] S3DistributionType set as FullyReplicated
[08:12:35] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-09-22:08:12:35:INFO] Determined delimiter of CSV input is ','
[08:12:35] S3DistributionType set as FullyReplicated
[08:12:35] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/va